In [1]:
from transformers import Pix2StructForConditionalGeneration, Pix2StructProcessor
from PIL import Image
import torch
import json
from tqdm import tqdm

/home/ges/anaconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import Pix2StructForConditionalGeneration, Pix2StructProcessor
from PIL import Image

model = Pix2StructForConditionalGeneration.from_pretrained("google/pix2struct-infographics-vqa-large").to("cuda")
processor = Pix2StructProcessor.from_pretrained("google/pix2struct-infographics-vqa-large")


In [11]:
model = Pix2StructForConditionalGeneration.from_pretrained("google/pix2struct-base")
# model.config
# model.lm_head = torch.nn.Linear(in_features=768, out_features=50354, bias=False)
print(model)
# # 저장된 state_dict 불러오기
model_path = "/home/ges/level3-cv-productserving-cv-10/Ko/pix2struct_base_custom.pt"
model.load_state_dict(torch.load(model_path))

# # GPU 사용 설정 (CUDA)
model.to("cuda")

Pix2StructForConditionalGeneration(
  (encoder): Pix2StructVisionModel(
    (embeddings): Pix2StructVisionEmbeddings(
      (patch_projection): Linear(in_features=768, out_features=768, bias=True)
      (row_embedder): Embedding(4096, 768)
      (column_embedder): Embedding(4096, 768)
      (dropout): Dropout(p=0.2, inplace=False)
    )
    (encoder): Pix2StructVisionEncoder(
      (layer): ModuleList(
        (0): Pix2StructVisionLayer(
          (attention): Pix2StructVisionAttention(
            (query): Linear(in_features=768, out_features=768, bias=False)
            (key): Linear(in_features=768, out_features=768, bias=False)
            (value): Linear(in_features=768, out_features=768, bias=False)
            (output): Linear(in_features=768, out_features=768, bias=False)
          )
          (mlp): Pix2StructVisionMlp(
            (wi_0): Linear(in_features=768, out_features=2048, bias=False)
            (wi_1): Linear(in_features=768, out_features=2048, bias=False)
         

RuntimeError: Error(s) in loading state_dict for Pix2StructForConditionalGeneration:
	size mismatch for decoder.embed_tokens.weight: copying a param with shape torch.Size([50354, 768]) from checkpoint, the shape in current model is torch.Size([50244, 768]).
	size mismatch for decoder.lm_head.weight: copying a param with shape torch.Size([50354, 768]) from checkpoint, the shape in current model is torch.Size([50244, 768]).

In [10]:
with open('/home/ges/level3-cv-productserving-cv-10/data/infographics/qas/infographicsVQA_test_v1.0.json', 'r') as file:
    data = json.load(file)

results = []

for item in tqdm(data['data'], desc="Generating answers", unit="qa"):
    image_path = item['image_local_name']
    question_id = item['questionId']
    question = item['question']
    
    
    # 이미지 로드 및 처리
    image = Image.open('/home/ges/level3-cv-productserving-cv-10/data/infographics/images/'+ image_path).convert("RGB")
    inputs = processor(images=image, text=question, return_tensors="pt").to("cuda")
    
    # 모델을 사용해 답변 생성
    predictions = model.generate(**inputs)
    answer = processor.decode(predictions[0], skip_special_tokens=True)
    
    
    print({"questionId": question_id, "answer": answer})
    # 결과 저장
    results.append({"questionId": question_id, "answer": answer})
    

# 결과를 JSON 파일로 저장
with open('result.json', 'w') as file:
    json.dump(results, file, ensure_ascii=False, indent=4)


Generating answers:   0%|          | 0/3288 [00:00<?, ?qa/s]

Generating answers:   0%|          | 1/3288 [00:00<40:02,  1.37qa/s]

{'questionId': 98304, 'answer': 'Trends'}


Generating answers:   0%|          | 2/3288 [00:01<42:13,  1.30qa/s]

{'questionId': 98309, 'answer': 'Twitter'}


...

{'questionId': 94435, 'answer': '14%'}


Generating answers: 100%|█████████▉| 3287/3288 [39:51<00:01,  1.17s/qa]

{'questionId': 98300, 'answer': '68%'}


Generating answers: 100%|██████████| 3288/3288 [39:52<00:00,  1.37qa/s]

{'questionId': 98301, 'answer': 'Technology'}


### Make eval matric with validatoin

In [24]:
import os
import glob

# 이미지가 있는 폴더 경로
image_path = '/home/ges/level3-cv-productserving-cv-10/data/infographics/images'

# 지원하는 이미지 파일 확장자 목록
image_extensions = ['*.jpg', '*.jpeg', '*.png']

# 이미지 파일 개수를 셀 변수
image_count = 0

# 각 확장자에 대해 폴더 내 파일 검색 및 개수 세기
for extension in image_extensions:
    image_count += len(glob.glob(os.path.join(image_path, extension)))

print(f'There are {image_count} image(s) in the folder.')


There are 5484 image(s) in the folder.
